***FUNCION DE LIMPIEZA***

In [1]:
import requests
import pandas as pd
import time
import random
from github import Github, GithubException
from datetime import datetime
import psycopg2
import os

In [2]:
#leer el csv en la carpeta actual
df_busqueda = pd.read_csv('guerrero.csv')
df_busqueda.head()

,id_tienda,sucursal,store_name,marca,producto,precio,contenido,busqueda,latitud,longitud,address,fecha_consulta
0,9993,Paradox Santa Fe,Soriana,1.2.3.,aceite vegetal comestible,46.0,botella 1 l,aceite,19.357303,-99.273539,"av sta fe 546, lomas de santa fe, contadero, c...",31/10/2023 18:41
1,9993,Paradox Santa Fe,Soriana,capullo,aceite de canola,60.5,botella 840 ml,aceite,19.357303,-99.273539,"av sta fe 546, lomas de santa fe, contadero, c...",31/10/2023 18:41
2,9993,Paradox Santa Fe,Soriana,oléico,aceite puro de cártamo,88.9,botella 825 ml,aceite,19.357303,-99.273539,"av sta fe 546, lomas de santa fe, contadero, c...",31/10/2023 18:41
3,9993,Paradox Santa Fe,Soriana,pam,aceite original comestible,73.9,aerosol 170 g,aceite,19.357303,-99.273539,"av sta fe 546, lomas de santa fe, contadero, c...",31/10/2023 18:41
4,9993,Paradox Santa Fe,Soriana,1.2.3.,aceite vegetal comestible,46.0,botella 1 l,aceite,19.357303,-99.273539,"av sta fe 546, lomas de santa fe, contadero, c...",31/10/2023 18:41


In [3]:
# #cambiar caracteres especiales en todo el df
# #Cambiar de Ã¡ a á
# df = df.replace({'Ã¡':'á'}, regex=True)
# #Cambiar de Ã© a é
# df = df.replace({'Ã©':'é'}, regex=True)
# #Cambiar de Ã­ a í
# df = df.replace({'Ã-­':'í'}, regex=True)
# #Cambiar de Ã³ a ó
# df = df.replace({'Ã³':'ó'}, regex=True)
# #Cambiar de Ãº a ú
# df = df.replace({'Ãº':'ú'}, regex=True)
# #Cambiar de Ã± a ñ
# df = df.replace({'Ã±':'ñ'}, regex=True)


In [3]:
reemplazos = {'Ã¡':'á', 'Ã©':'é', 'Ã-­':'í', 'Ã³':'ó', 'Ãº':'ú', 'Ã±':'ñ'}
df_busqueda = df_busqueda.replace(reemplazos, regex=True)

In [4]:
#Contar los datos por tipo en la columna busqueda
df_busqueda['busqueda'].value_counts()


busqueda
desodorante                  452
aceite                       437
azúcar                       435
sal                          429
fibra                        419
rastillos                    404
arroz                        383
franelas                     366
atún                         363
jabón+de+barra               360
toallas+femeninas            280
pasta+dental                 257
frijol                       247
jabón+en+polvo               217
sardina                      216
cepillos+dentales            208
mayonesa                     201
cloro                        187
limpiador+para+pisos         186
sopas+instantánea            177
mermelada                    166
lata+de+chiles               160
rollos+de+papel+higiénico    149
frasco+café+soluble          141
escobas                      129
chocolate+en+polvo           108
toallas+húmedas               83
peine                         77
toallas+faciales              35
harina+para+atole             30
j

In [45]:
#Función de limpieza de prueba
def limpiaprod(df):
    #Aceite
    #aceite = df[df['producto'].str.contains("aceite") & (df['contenido'].str.contains("946"))]
    #aceite = aceite[~aceite["producto"].str.contains("oliva")]

    #Arroz
    #arroz = df[df['producto'].str.contains("arroz") & (df['contenido'].str.contains("1 kg"))]
    #arroz = arroz[~arroz["producto"].str.contains("sushi") & ~arroz["producto"].str.contains("arborio") & ~arroz["producto"].str.contains("precocido")& ~arroz["producto"].str.contains("orgánico")]

    #Atún
    #atún = df[df['producto'].str.contains("atún") & (df['contenido'].str.contains("140")) & (df['contenido'].str.contains("lata")) & (df['producto'].str.contains("agua"))]
    #atún = atún[~atún["producto"].str.contains("aceite")]

    #Azúcar
    azúcar = df[df['producto'].str.contains("azúcar") & (df['contenido'].str.contains("2 kg"))]
                  
    #aceite=aceite.query('producto.str.contains("vegetal") | producto.str.contains("canola")')
    #aceite = aceite[~aceite["producto"].str.contains("acondicionador") & ~aceite["contenido"].str.contains("g")]

    #Concatenar
    #finaldf = pd.concat([aceite, arroz, atún, azúcar])
    finaldf = pd.concat([azúcar])
    #reset index
    finaldf.reset_index(drop=True, inplace=True)
    return finaldf


In [46]:
limpiaprod(df_busqueda)

,id_tienda,sucursal,store_name,marca,producto,precio,contenido,busqueda,latitud,longitud,address,fecha_consulta
0,9993,Paradox Santa Fe,Soriana,dolores,lomo de atún en agua premium,37.5,lata 140 g,atún,19.357303,-99.273539,"av sta fe 546, lomas de santa fe, contadero, c...",31/10/2023 18:41
1,9993,Paradox Santa Fe,Soriana,dolores,atún en hojuelas en agua,21.5,lata 140 g,atún,19.357303,-99.273539,"av sta fe 546, lomas de santa fe, contadero, c...",31/10/2023 18:41
2,9993,Paradox Santa Fe,Soriana,tuny,atún en hojuela agua clásico,21.5,lata 140 g,atún,19.357303,-99.273539,"av sta fe 546, lomas de santa fe, contadero, c...",31/10/2023 18:41
3,9993,Paradox Santa Fe,Soriana,fresh label,atún en agua,11.0,lata 140 g,atún,19.357303,-99.273539,"av sta fe 546, lomas de santa fe, contadero, c...",31/10/2023 18:41
4,4449,La Comer Tlalpan,La Comer,dolores,lomo de atún en agua premium,35.9,lata 140 g,atún,19.288687,-99.179181,"plazuela de la fama 1, la fama, 14410 tlalpan,...",31/10/2023 18:41
5,4449,La Comer Tlalpan,La Comer,tuny,lomo atún aleta amarilla trozos en agua light,26.0,lata 140 g,atún,19.288687,-99.179181,"plazuela de la fama 1, la fama, 14410 tlalpan,...",31/10/2023 18:41
6,9991,La Villa,Soriana,dolores,lomo de atún en agua premium,36.9,lata 140 g,atún,19.465417,-99.125438,"calz de guadalupe 192, vallejo, gustavo a. mad...",31/10/2023 18:41
7,9991,La Villa,Soriana,dolores,atún en hojuelas en agua,21.5,lata 140 g,atún,19.465417,-99.125438,"calz de guadalupe 192, vallejo, gustavo a. mad...",31/10/2023 18:41
8,9991,La Villa,Soriana,tuny,atún en hojuela agua clásico,21.5,lata 140 g,atún,19.465417,-99.125438,"calz de guadalupe 192, vallejo, gustavo a. mad...",31/10/2023 18:41
9,9991,La Villa,Soriana,fresh label,atún en agua,11.0,lata 140 g,atún,19.465417,-99.125438,"calz de guadalupe 192, vallejo, gustavo a. mad...",31/10/2023 18:41


In [83]:
x["contenido"].unique()

array(['botella 1 l', 'botella 840 ml', 'botella 825 ml', 'aerosol 170 g',
       'botella 850 ml', 'botella 765 ml', 'botella 845 ml',
       'spray 170 g', 'aceite 825 ml', 'botella 250 ml', '825 ml',
       'botella 500 ml', 'aerosol 141 g', 'bote 420 ml', 'spray 180 ml',
       '145 ml', '170 g', '473 ml', '250 g', '250 ml', '450 ml', '280 ml',
       'frasco 235 ml', 'botella 800 ml', 'botella 750 ml', 'spray 141 g',
       '500 ml', 'lata 950 ml', '750 ml', '946 ml', 'spray 145 ml',
       'garcia de la cruz 250 ml.', 'tassos 500 ml.', '141 g',
       'botella 100 ml', 'botella 200 ml', 'botella 130 ml', 'pouch 78 g',
       'caja 140 g', 'atún 295 g', 'caja 112 g', 'lata 130 g', '120 g',
       'lata 140 g', 'caja 120 g', '140 g', '112 g', 'sobre 74 g',
       'lata 295 g', '400 g', '130 g', '115 g', 'caja 115 g',
       'lata 120 g', 'lata 280 g', 'frasco 270 g', 'botella 190 ml',
       'botella 2 l', 'botella 760 ml', '591 ml', '385 ml',
       'botella 385 ml', 'botella 650 

In [31]:
#Contar los valores de busqueda = aceite
df_prod_limpia['busqueda'].value_counts()

aceite    427
Name: busqueda, dtype: int64

In [ ]:
# #Función para obtener los productos deseados
# def limpiaprod(df):
#     #Tortilla.
#     tortilla = df.query('busqueda.str.contains("tortilla") and contenido.str.contains("precio por kg")')
#     tortilla = tortilla[~tortilla["producto"].str.contains("masa")]
#     #Aceite
     aceite = df.query('busqueda.str.contains("aceite") & (contenido.str.contains("850") | contenido.str.contains("875") | contenido.str.contains("900") | contenido.str.contains("946") | contenido.str.contains("1 l"))')
#     aceite=aceite.query('producto.str.contains("vegetal") | producto.str.contains("canola")')
#     #Arroz
#     arroz = df.query('busqueda.str.contains("arroz") & (contenido.str.contains("a granel") | contenido.str.contains("1 kg") | contenido.str.contains("900"))')
#     #Atún
#     atun= df.query('busqueda.str.contains("atún") & contenido.str.contains("lata") & (contenido.str.contains("125") | contenido.str.contains("130") | contenido.str.contains("135") | contenido.str.contains("140") | contenido.str.contains("142") | contenido.str.contains("145") | contenido.str.contains("148") | contenido.str.contains("145") | contenido.str.contains("150"))')
#     atun=atun.query('producto.str.contains("agua") | producto.str.contains("aceite")')
#     #Cebolla
#     cebolla = df.query('busqueda.str.contains("cebolla") & contenido.str.contains("precio por kg")')
#     cebolla = cebolla.query('producto.str.contains("blanca")')
#     #Chuleta
#     chuleta = df.query('busqueda.str.contains("chuleta") & (contenido.str.contains("precio por kg") | contenido.str.contains("a granel"))')
#     #Chile jalapeño
#     chile = df.query('busqueda.str.contains("chile") & contenido.str.contains("precio por kg") & ~(producto.str.contains("pastel"))')
#     #Bistec
#     bistec = df.query('busqueda.str.contains("bistec") & contenido.str.contains("precio por kg")')
#     #Frijol
#     frijol= df.query('busqueda.str.contains("frijol") & (contenido.str.contains("800") or contenido.str.contains("820") | contenido.str.contains("840") | contenido.str.contains("900") | contenido.str.contains("907") | contenido.str.contains("908") | contenido.str.contains("kg") | contenido.str.contains("145"))')
#     frijol = frijol.query('producto.str.contains("negro")')
#     #Huevo
#     huevo=df.query('busqueda.str.contains("huevo") & (producto.str.contains("blanco") | producto.str.contains("rojo")) & (contenido.str.contains("18") | contenido.str.contains("12") | contenido.str.contains("30"))')
#     huevo = huevo[~huevo["producto"].str.contains("yoghurt")]
#     #jitomate
#     jitomate=df.query('busqueda.str.contains("jitomate") & (contenido.str.contains("precio por kg") | contenido.str.contains("1 kg"))')
#     jitomate = jitomate.query('producto.str.contains("saladet")')
#     #Limón
#     limon = df.query('busqueda.str.contains("limón") & (contenido.str.contains("precio por kg") | contenido.str.contains("1 kg"))')
#     limon = limon[~limon["contenido"].str.contains("2.1")]
#     limon =limon.query('producto.str.contains("persa") | producto.str.contains("semilla") | producto.str.contains("agrio")')
#     #Manzana
#     manzana = df.query('busqueda.str.contains("manzana") & (contenido.str.contains("precio por kg") | contenido.str.contains("1 kg"))')
#     manzana = manzana.query('producto.str.contains("golden")')
#     #Naranja
#     naranja = df.query('busqueda.str.contains("naranja") & contenido.str.contains("precio por kg")')
#     naranja = naranja.query('producto.str.contains("valencia")')
#     #papa
#     papa = df.query('busqueda.str.contains("papa") & (contenido.str.contains("precio por kg") | contenido.str.contains("1 kg"))')
#     papa=papa.query('producto.str.contains("blanca") | producto.str.contains("alfa")')
#     #Pollo
#     pollo = df.query('busqueda.str.contains("pollo") & (contenido.str.contains("precio por kg") | contenido.str.contains("1 kg"))')
#     pollo = pollo.query('producto.str.contains("entero")')
#     pollo = pollo[~pollo["producto"].str.contains("orgánico")]
#     #Sardina
#     sardina = df.query('busqueda.str.contains("sardina") & producto.str.contains("tomate")')
#     sardina =sardina.query('contenido.str.contains("380") | contenido.str.contains("425")')
#     #zanahoria
#     zanahoria=df.query('busqueda.str.contains("zanahoria") & contenido.str.contains("precio por kg")')
#     zanahoria = zanahoria[~zanahoria["producto"].str.contains("cambray")]
#     #azúcar
#     azucar= df.query('busqueda.str.contains("azúcar") & (contenido.str.contains("900 g") | contenido.str.contains("950") | contenido.str.contains("1 kg"))')
#     azucar=azucar.query('producto.str.contains("morena") | producto.str.contains("estándar")')
#     azucar = azucar[~azucar["producto"].str.contains("orgánico")]
#     #jabón
#     jabon=df.query('busqueda.str.contains("jabón") & producto.str.contains("tocador") & contenido.str.contains("barra")')
#     jabon = jabon[~jabon["contenido"].str.contains("x") & ~jabon["contenido"].str.contains("90") &
#                    ~jabon["contenido"].str.contains("200") & ~jabon["contenido"].str.contains("pieza")]
#     #leche
#     leche=df.query('busqueda.str.contains("leche")')
#     leche=leche.query('producto.str.contains("pasteurizada") | producto.str.contains("entera") | producto.str.contains("ultrapasteurizada")')
#     leche=leche.query('contenido.str.contains("1 l")')
#     leche = leche[~leche["contenido"].str.contains("x") & ~leche["producto"].str.contains("orgánica") &
#                    ~leche["producto"].str.contains("deslactosada") & ~leche["producto"].str.contains("light")]
#     #pan
#     pan=df.query('busqueda.str.contains("pan")')
#     pan=pan.query('producto.str.contains("blanco") | producto.str.contains("integral")')
#     pan=pan.query('contenido.str.contains("g") | contenido.str.contains("grs")')
#     pan=pan[~pan["contenido"].str.contains("100") & 
#             ~pan["contenido"].str.contains("120") & ~pan["contenido"].str.contains("250") &
#             ~pan["contenido"].str.contains("255") & ~pan["contenido"].str.contains("260") &
#             ~pan["contenido"].str.contains("360") & ~pan["contenido"].str.contains("460") &
#             ~pan["contenido"].str.contains("305") & ~pan["contenido"].str.contains("450") &
#             ~pan["contenido"].str.contains("454") & ~pan["contenido"].str.contains("300") &
#             ~pan["contenido"].str.contains("480") & ~pan["contenido"].str.contains("425") &
#             ~pan["contenido"].str.contains("567") & ~pan["contenido"].str.contains("580") &
#                 ~pan["contenido"].str.contains("500")]
#     #papel
#     papel=df.query('busqueda.str.contains("papel") & contenido.str.contains("4")')
#     papel=papel[~papel["contenido"].str.contains("24") & ~papel["contenido"].str.contains("42") &
#                 ~papel["contenido"].str.contains("40") & ~papel["contenido"].str.contains("420")]
#     #pasta
#     pasta=df.query('busqueda.str.contains("pasta")')
#     pasta=pasta.query('contenido.str.contains("g") | contenido.str.contains("grs")')
#     pasta = pasta[~pasta["contenido"].str.contains("x")]
#     pasta=pasta.query('producto.str.contains("pasta") | producto.str.contains("sopa")')
#     pasta=pasta.query('contenido.str.contains("200") | contenido.str.contains("205") | contenido.str.contains("220") | contenido.str.contains("227")')
#     #plátano
#     #sniim solo considera tabasco
#     platano=df.query('busqueda.str.contains("plátano") & contenido.str.contains("precio por kg") & (producto.str.contains("tabasco") | producto.str.contains("chiapas"))')
#     #Lechuga
#     lechuga=df.query('busqueda.str.contains("lechuga") & (producto.str.contains("romana") | producto.str.contains("orejona")) & contenido.str.contains("1 pieza")')
#     #espinaca
#     espinaca=df.query('busqueda.str.contains("espinaca") & (contenido.str.contains("rollo") | contenido.str.contains("manojo") | contenido.str.contains("pieza"))')
#     espinaca=espinaca[~espinaca["contenido"].str.contains("bolsa") & ~espinaca["contenido"].str.contains("domo")]
#     espinaca=espinaca[~espinaca["producto"].str.contains("pollo")]
#     #lenteja
#     lenteja=df.query('busqueda.str.contains("lenteja") & contenido.str.contains("bolsa")')
#     lenteja=lenteja[~lenteja["contenido"].str.contains("113")]
#     lenteja =lenteja.query('contenido.str.contains("g") | contenido.str.contains("grs")')
#     #Jamón
#     jamon=df.query('busqueda.str.contains("jamón")')
#     jamon =jamon.query('contenido.str.contains("granel") | contenido.str.contains("1 kg")')
#     jamon =jamon.query('producto.str.contains("pavo") | contenido.str.contains("virginia") | contenido.str.contains("pechuga")')
#     #Mojarra
#     mojarra = df.query('busqueda.str.contains("mojarra") & (contenido.str.contains("precio por kg") | contenido.str.contains("1 kg"))')
#     #puré de tomate
#     pure=df.query('busqueda.str.contains("puré") & contenido.str.contains("kg")')
#     #servilleta
#     servilleta = df.query('busqueda.str.contains("servilleta") & (contenido.str.contains("180") | contenido.str.contains("200"))')
#     #toalla femenina
#     toalla = df.query('busqueda.str.contains("toalla") & (contenido.str.contains("8") | contenido.str.contains("10"))')
#     toalla=toalla[~toalla["contenido"].str.contains("28") & ~toalla["contenido"].str.contains("18") &
#                 ~toalla["contenido"].str.contains("48")]


#     #Concatenar
#     finaldf = pd.concat([tortilla, aceite, 
#                          arroz, atun, cebolla, chuleta, chile,
#                          bistec, frijol,huevo,jitomate,limon,manzana,
#                          naranja,papa,pollo,sardina, zanahoria,azucar,
#                          jabon,leche,pan,papel,pasta,platano,lechuga,
#                          espinaca,lenteja,jamon,mojarra,pure,servilleta,
#                          toalla])
#     #reset index
#     finaldf.reset_index(drop=True, inplace=True)
#     return finaldf